In [1]:
import os
import pandas as pd
import polars as pl
from pathlib import Path

data_root = Path(
    "/Users/larsankile/Library/CloudStorage/GoogleDrive-lars.ankile@gmail.com/My Drive/Datasets/defi-measurement/mev-boost/"
)

In [2]:
df = pd.read_parquet(data_root / "openethdata_eth_data.parquet.gzip")
df = df[~df.block_number.isna()]

df = df.astype(
    {
        "block_number": "int64",
        "relay": "string[pyarrow]",
        "builder_pubkey": "string[pyarrow]",
        "proposer_pubkey": "string[pyarrow]",
        "mevboost_value": "float64",
        "builder": "string[pyarrow]",
        "validator": "string[pyarrow]",
    }
)

In [3]:
df = df.sort_values("block_number")

# Filter out all transactions that are 'missed'
df = df[df.proposer_pubkey != "missed"]
df = df[~df.block_number.isna()]
df = df[~df.relay.isnull()]

# Take out rows where block number is 0
df = df[df.block_number != 0]

df = df.assign(mevboost_value=df.mevboost_value.astype(float) / 10**18)

print(df.shape)

df.head()

(3147780, 9)


,date,slot,block_number,relay,builder_pubkey,proposer_pubkey,mevboost_value,builder,validator
971927,2022-09-15 08:33:47,4700567,15537940,flashbots,0xa1dead01e65f0a0eee7b5170223f20c8f0cbf122eac3...,0xa606aca2180226fc69e0652134eca4c7f4a428585c11...,0.283204,Flashbots,0xa606aca2180226fc69e0652134eca4c7f4a428585c11...
760652,2022-09-15 08:35:23,4700575,15537948,flashbots,0xa1dead01e65f0a0eee7b5170223f20c8f0cbf122eac3...,0x9489e9354f515580f6a949f69eed311eeb26b268b744...,0.845802,Flashbots,0x9489e9354f515580f6a949f69eed311eeb26b268b744...
325728,2022-09-15 08:41:35,4700606,15537978,flashbots,0x81beef03aafd3dd33ffd7deb337407142c80fea2690e...,0x82d5ebc6c3ef5eda41dcbb1cbdfd5863ab6e4d16b137...,0.213524,Flashbots,stakewise
1498601,2022-09-15 08:59:11,4700694,15538064,flashbots,0xa1dead01e65f0a0eee7b5170223f20c8f0cbf122eac3...,0x8e7d5eed0f72cd56e547e2fabea19a65668577fd2d64...,0.113136,Flashbots,0x8e7d5eed0f72cd56e547e2fabea19a65668577fd2d64...
99589,2022-09-15 09:05:11,4700724,15538094,flashbots,0xa1dead01e65f0a0eee7b5170223f20c8f0cbf122eac3...,0xae1426c22c5c0d91d62c5a224759f44569a0a52e6cb8...,0.637987,Flashbots,rocketpool


In [4]:
# Convert the
df[df.mevboost_value.isnull()].shape

(0, 9)

In [5]:
# Convert to polars to ensure it works and then write a new file back to parquet
df = pl.DataFrame(df)

df.write_parquet(data_root / "openethdata_eth_data_clean.parquet", compression="gzip")

In [2]:
# Check we can read it again
df = pd.read_parquet(data_root / "openethdata_eth_data_clean.parquet")

df.head()

,date,slot,block_number,relay,builder_pubkey,proposer_pubkey,mevboost_value,builder,validator
0,2022-09-15 08:33:47,4700567,15537940,flashbots,0xa1dead01e65f0a0eee7b5170223f20c8f0cbf122eac3...,0xa606aca2180226fc69e0652134eca4c7f4a428585c11...,0.283204,Flashbots,0xa606aca2180226fc69e0652134eca4c7f4a428585c11...
1,2022-09-15 08:35:23,4700575,15537948,flashbots,0xa1dead01e65f0a0eee7b5170223f20c8f0cbf122eac3...,0x9489e9354f515580f6a949f69eed311eeb26b268b744...,0.845802,Flashbots,0x9489e9354f515580f6a949f69eed311eeb26b268b744...
2,2022-09-15 08:41:35,4700606,15537978,flashbots,0x81beef03aafd3dd33ffd7deb337407142c80fea2690e...,0x82d5ebc6c3ef5eda41dcbb1cbdfd5863ab6e4d16b137...,0.213524,Flashbots,stakewise
3,2022-09-15 08:59:11,4700694,15538064,flashbots,0xa1dead01e65f0a0eee7b5170223f20c8f0cbf122eac3...,0x8e7d5eed0f72cd56e547e2fabea19a65668577fd2d64...,0.113136,Flashbots,0x8e7d5eed0f72cd56e547e2fabea19a65668577fd2d64...
4,2022-09-15 09:05:11,4700724,15538094,flashbots,0xa1dead01e65f0a0eee7b5170223f20c8f0cbf122eac3...,0xae1426c22c5c0d91d62c5a224759f44569a0a52e6cb8...,0.637987,Flashbots,rocketpool
